In [9]:
from netCDF4 import Dataset
import numpy as np
from tqdm.notebook import tqdm
import datetime as dt
import os
import json

In [ ]:
def split_data_mask(variable_name) -> dict:
    
    var = D[variable_name]
    masked_array = var[:]
    data = masked_array.data.tolist()
    mask = masked_array.mask.tolist()
    
    dic = {'data' : data, 'mask' : mask}
    
    return dic

months = range(1,13)
num_days = [31,28,31,30,31,30,31,31,30,31,30,31]

year_dic = dict()
month_dic = dict()
day_dic = dict()
except_date = []

for month, num_days in zip(months, num_days):
    for day in tqdm(range(1,num_days+1)):
        for year in range(1981,2022):
            if (year == 1981) and (month in [1,2,3,4,5,6,7,8]) : 
                continue
            
            date = dt.date(year,month,day).strftime('%Y%m%d')
            
            try :
                if year < 2016 : 
                    D = Dataset(f'/Volumes/T7/AVHRR_OI_SST/v2.1/{year}/oisst-avhrr-v02r01.{date}.nc', 'r', format='NETCDF4')
                    variables = ['anom', 'err', 'ice', 'lat', 'lon', 'sst', 'time', 'zlev']

                else :
                    if os.path.exists(f'/Volumes/T7/AVHRR_OI_SST/v2.1/{year}/{date}120000-NCEI-L4_GHRSST-SSTblend-AVHRR_OI-GLOB-v02.0-fv02.1.nc'):
                        D = Dataset(f'/Volumes/T7/AVHRR_OI_SST/v2.1/{year}/{date}120000-NCEI-L4_GHRSST-SSTblend-AVHRR_OI-GLOB-v02.0-fv02.1.nc', 'r', format='NETCDF4')
                    else :
                        D = Dataset(f'/Volumes/T7/AVHRR_OI_SST/v2.1/{year}/{date}120000-NCEI-L4_GHRSST-SSTblend-AVHRR_OI-GLOB-v02.1-fv02.1.nc', 'r', format='NETCDF4')

                    variables = ['lat', 'lon', 'time', 'lat_bnds', 'lon_bnds', 'analysed_sst', 'analysis_error', 'mask', 'sea_ice_fraction']

                # print(type(anom[:])) # type = MaskedArray

                vars_dic = dict()
                for variable in variables : 
                    data_mask_dic = split_data_mask(variable)
                    vars_dic[variable] = data_mask_dic

                D.close()
            
            except :
                except_date.append(date)
                vars_dic = 'NULL'

            year_dic[year] = vars_dic
        day_dic[day] = year_dic
    month_dic[month] = day_dic

  0%|          | 0/31 [00:00<?, ?it/s]

  0%|          | 0/28 [00:00<?, ?it/s]

In [8]:
except_date

['20210116']

In [10]:
json_file = json.dumps(month_dic)

TypeError: Object of type ndarray is not JSON serializable